In [4]:
import pandas as pd
import requests
import json # Importa el módulo 'json'

def obtener_datos(series_id, site_code):
    """
    Realiza una petición a la API y devuelve un DataFrame.
    """
    url = f"https://alerta.ina.gob.ar/pub/datos/datos&timeStart=2025-09-18&timeEnd=2025-09-20&seriesId={series_id}&siteCode={site_code}&varId=2&format=csv"
    
    try:
        # Lee el CSV directamente desde la URL.
        df = pd.read_csv(url, comment='#', sep=',', header=None)
        df.columns = ['obs_id', 'timestart', 'timeend', 'valor', 'timeupdate']
        
        # Agrega las columnas adicionales
        df['seriesId'] = series_id
        df['siteCode'] = site_code
        
        return df
    
    except Exception as e:
        print(f"Error al obtener datos para seriesId {series_id}: {e}")
        return pd.DataFrame() # Devuelve un DataFrame vacío en caso de error

# Leer el archivo JSON de forma síncrona con Python
try:
    with open("alturasHidrometricas.json", 'r', encoding='utf-8') as file:
        alturas_json = json.load(file)
except FileNotFoundError:
    print("Error: El archivo 'alturasHidrometricas.json' no se encontró.")
    alturas_json = []

lista_de_dataframes = []

for item in alturas_json:
    df_temporal = obtener_datos(item['seriesid'], item['sitecode'])
    lista_de_dataframes.append(df_temporal)

# Concatena todos los DataFrames en uno solo
df_final = pd.concat(lista_de_dataframes, ignore_index=True)

# Muestra las primeras 5 filas del DataFrame final
print(df_final.head())

# Opcional: Guarda el DataFrame final en un solo archivo CSV
df_final.to_csv("datos_finales.csv", index=False)

Error al obtener datos para seriesId 55: No columns to parse from file
Error al obtener datos para seriesId 151: No columns to parse from file
Error al obtener datos para seriesId 3279: No columns to parse from file
Error al obtener datos para seriesId 3280: 'utf-8' codec can't decode byte 0xf3 in position 695: invalid continuation byte
Error al obtener datos para seriesId 3309: 'utf-8' codec can't decode byte 0xe1 in position 759: invalid continuation byte
Error al obtener datos para seriesId 3312: 'utf-8' codec can't decode byte 0xe1 in position 388: invalid continuation byte
Error al obtener datos para seriesId 3314: 'utf-8' codec can't decode byte 0xe1 in position 417: invalid continuation byte
Error al obtener datos para seriesId 3345: 'utf-8' codec can't decode byte 0xf3 in position 560: invalid continuation byte
Error al obtener datos para seriesId 3347: 'utf-8' codec can't decode byte 0xf3 in position 334: invalid continuation byte
Error al obtener datos para seriesId 6060: 'ut

In [9]:
df_nuevito = pd.read_csv("datos_finales.csv")
df_nuevito.seriesId.value_counts()

seriesId
35411    530
35405    529
35407    529
35409    524
3307     175
        ... 
12         2
8          2
10         2
37076      1
40         1
Name: count, Length: 397, dtype: int64